# Geospatial Image Analysis with Moondream Vision Language Model

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/moondream.ipynb)

This notebook demonstrates how to use the [Moondream](https://moondream.ai) vision language model for geospatial image analysis. Moondream is a small but powerful vision language model that can:

- **Caption**: Generate descriptions of satellite/aerial imagery
- **Query**: Answer questions about image content
- **Detect**: Locate objects with bounding boxes
- **Point**: Find specific objects and return their coordinates

The GeoAI integration adds support for GeoTIFF files with automatic georeferencing of outputs.


## Install packages

Uncomment the following line to install the required packages.

In [ ]:
# %pip install -U geoai-py

## Import libraries

In [ ]:
import leafmap
from geoai import MoondreamGeo
import geoai

## Download sample data

We'll use a sample GeoTIFF image of a parking lot with buildings and vegetation.

In [ ]:
url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/parking_lot.tif"
image_path = geoai.download_file(url)
image_path

## Visualize the image

Let's first visualize the sample image on an interactive map.

In [ ]:
m = leafmap.Map()
m.add_raster(image_path, layer_name="Satellite Image")
m

## Initialize the Moondream processor

Load the Moondream2 model. The first time you run this, the model will be downloaded from HuggingFace (~3.7GB).

**Note**: For reproducibility, we specify a specific model revision date.

In [ ]:
processor = MoondreamGeo(
    model_name="vikhyatk/moondream2", revision="2025-06-21", device="cuda"
)

## Image Captioning

Generate a description of the satellite image. The `length` parameter controls the detail level: "short", "normal", or "long".

In [ ]:
result = processor.caption(image_path, length="normal")
print(result["caption"])

## Visual Question Answering

Ask questions about the image content and get natural language answers.

In [ ]:
result = processor.query("How many buildings are in the image?", image_path)
print(result["answer"])

In [ ]:
result = processor.query("What are the building roof colors?", image_path)
print(result["answer"])

## Object Detection

Detect objects and get bounding boxes. Results are automatically georeferenced when using GeoTIFF input.

### Detect buildings

In [ ]:
result = processor.detect(image_path, "building", output_path="buildings.geojson")
print(f"Detected {len(result['objects'])} buildings")

View the GeoDataFrame with georeferenced bounding boxes:

In [ ]:
result["gdf"]

Add the detected buildings to the map:

In [ ]:
m.add_gdf(result["gdf"], layer_name="buildings")
m

## Point Localization

Find object center points instead of bounding boxes. Useful for counting and locating objects.

### Find building centroids

In [ ]:
result = processor.point(
    image_path, "building", output_path="building_centroids.geojson"
)
print(f"Found {len(result['points'])} building centroids")

In [ ]:
m.add_gdf(result["gdf"], layer_name="building_centroids")

### Detect trees

In [ ]:
result = processor.detect(image_path, "tree", output_path="trees.geojson")
print(f"Detected {len(result['objects'])} trees")

In [ ]:
m.add_gdf(result["gdf"], layer_name="trees")

### Find tree centroids

In [ ]:
result = processor.point(image_path, "trees", output_path="tree_centroids.geojson")
print(f"Found {len(result['points'])} tree centroids")

In [ ]:
m.add_gdf(result["gdf"], layer_name="tree_centroids")

## Display final map

View all detected objects and centroids on the map.


In [ ]:
m

![](https://github.com/user-attachments/assets/032daf91-1cb3-464b-9576-17f61767ab6f)